# 1. 모듈 Import 및 데이터 불러오기

In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import mean_squared_error

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
train_data = pd.read_csv('../data/최종데이터/for_train_data.csv')
test_data = pd.read_csv('../data/최종데이터/for_test_data.csv')
submission = pd.read_csv('./data/최종데이터/second_group_submission.csv', encoding='cp949')

In [164]:
train_data.head(3)

,term,emd_nm,em_g_gap,em_g_gap_rate,em_g_avg,em_g_std,resid_reg_pop_gap,resid_reg_pop_gap_rate,resid_reg_pop_avg,resid_reg_pop_std,...,deli_per_use_amt_std,food_per_use_amt_gap,food_per_use_amt_gap_rate,food_per_use_amt_avg,food_per_use_amt_std,temperature,rain,fine_dust,em_g,peak
0,2018-06-01,건입동,3468900,1.016998,45007516,1486514.83,31.0,0.998265,9799.000000,12.68,...,395.39,10855.341526,1.069767,53964.320945,4530.37,21.721667,8.994167,31.633333,48187900,0
1,2018-06-01,남원읍,5900300,1.012975,40838033,2661150.77,66.0,1.003453,19144.000000,26.98,...,367.26,7150.172638,1.044615,34163.932414,3094.34,21.721667,8.994167,31.633333,42287450,0
2,2018-06-01,노형동,24880750,1.013611,259129650,10977885.73,52.0,0.999026,53351.666667,24.51,...,258.08,7942.755643,0.963592,39179.948301,3467.90,21.721667,8.994167,31.633333,272193365,0


In [165]:
test_data.head(3)

,term,emd_nm,em_g_gap,em_g_gap_rate,em_g_avg,em_g_std,resid_reg_pop_gap,resid_reg_pop_gap_rate,resid_reg_pop_avg,resid_reg_pop_std,...,deli_per_use_amt_avg,deli_per_use_amt_std,food_per_use_amt_gap,food_per_use_amt_gap_rate,food_per_use_amt_avg,food_per_use_amt_std,temperature,rain,fine_dust,peak
0,2021-07-01,건입동,5989000,1.203722,32984150,2584274.49,67.0,0.993099,8941.000000,30.47,...,23822.779936,266.01,2663.530528,0.950798,43424.033709,1164.63,26.918548,6.504839,20.053763,8
1,2021-07-01,구좌읍,6452450,1.241856,30243200,2677238.86,118.0,0.993239,15455.666667,52.83,...,22662.817683,254.59,4771.683809,0.882261,32689.677020,2127.68,26.918548,6.504839,20.053763,8
2,2021-07-01,남원읍,6423350,1.102866,60246216,2888106.38,47.0,0.998827,18728.000000,19.20,...,21800.036803,138.04,3201.393896,0.946122,36743.632597,1320.03,26.918548,6.504839,20.053763,8


# 2. 19년부터 시행된 4개지역(구좌읍/조천읍/한경면/한림읍)만 포함

In [166]:
train_data = train_data[train_data.emd_nm.isin(['구좌읍', '조천읍', '한경면', '한림읍'])]
test_data = test_data[test_data.emd_nm.isin(['구좌읍', '조천읍', '한경면', '한림읍'])]

In [167]:
test_data

,term,emd_nm,em_g_gap,em_g_gap_rate,em_g_avg,em_g_std,resid_reg_pop_gap,resid_reg_pop_gap_rate,resid_reg_pop_avg,resid_reg_pop_std,...,deli_per_use_amt_avg,deli_per_use_amt_std,food_per_use_amt_gap,food_per_use_amt_gap_rate,food_per_use_amt_avg,food_per_use_amt_std,temperature,rain,fine_dust,peak
1,2021-07-01,구좌읍,6452450,1.241856,30243200,2677238.86,118.0,0.993239,15455.666667,52.83,...,22662.817683,254.59,4771.683809,0.882261,32689.677020,2127.68,26.918548,6.504839,20.053763,8
33,2021-07-01,조천읍,7040900,1.122817,56984083,3188248.58,83.0,0.998937,25370.333333,34.57,...,24375.140323,490.66,2565.188051,0.953747,34053.564799,1060.69,26.918548,6.504839,20.053763,8
38,2021-07-01,한경면,5631450,1.325956,20397416,2339140.44,45.0,1.002314,9075.000000,18.38,...,24045.823727,1154.55,5937.848008,0.921502,35129.039909,2424.14,26.918548,6.504839,20.053763,8
39,2021-07-01,한림읍,13703066,1.228185,67862894,5756246.23,53.0,0.997978,21235.000000,22.99,...,24923.926013,443.40,5283.390710,0.887778,43724.732863,2381.22,26.918548,6.504839,20.053763,8
43,2021-08-01,구좌읍,3531850,1.114228,32833966,1457124.74,23.0,0.999351,15413.000000,9.42,...,23611.717490,1207.47,1143.007105,1.036980,31480.118118,466.63,26.664516,9.360484,17.913978,21
75,2021-08-01,조천읍,715900,0.987974,59092416,312806.95,103.0,1.004067,25377.000000,42.10,...,24113.573288,536.41,925.280866,1.011515,33324.746793,379.81,26.664516,9.360484,17.913978,21
80,2021-08-01,한경면,5920200,1.281815,23614383,2467963.55,98.0,1.010826,9099.666667,40.05,...,22852.756142,653.02,2946.659446,1.009198,33248.387202,1324.85,26.664516,9.360484,17.913978,21
81,2021-08-01,한림읍,9515000,1.136356,74277233,3901955.26,31.0,0.999010,21210.333333,12.92,...,25025.250855,411.07,2210.869715,1.040411,42700.559198,946.41,26.664516,9.360484,17.913978,21


# 3. gap_rate, std가 이름에 포함된 변수삭제

In [168]:
col_list = list(train_data.columns)
col_list

['term',
 'emd_nm',
 'em_g_gap',
 'em_g_gap_rate',
 'em_g_avg',
 'em_g_std',
 'resid_reg_pop_gap',
 'resid_reg_pop_gap_rate',
 'resid_reg_pop_avg',
 'resid_reg_pop_std',
 'foreign_pop_gap_rate',
 'foreign_pop_std',
 'total_pop_gap',
 'total_pop_gap_rate',
 'total_pop_avg',
 'total_pop_std',
 'f_resid_reg_rate_gap',
 'f_resid_reg_rate_gap_rate',
 'f_resid_reg_rate_avg',
 'f_resid_reg_rate_std',
 'f_foreign_rate_gap',
 'f_foreign_rate_gap_rate',
 'f_foreign_rate_avg',
 'f_foreign_rate_std',
 'f_total_rate_gap',
 'f_total_rate_gap_rate',
 'f_total_rate_avg',
 'f_total_rate_std',
 'floating_resd_rate_gap',
 'floating_resd_rate_gap_rate',
 'floating_resd_rate_avg',
 'floating_resd_rate_std',
 'floating_pop_cnt_gap',
 'floating_pop_cnt_gap_rate',
 'floating_pop_cnt_avg',
 'floating_pop_cnt_std',
 'floating_visit_rate_1020_gap',
 'floating_visit_rate_1020_gap_rate',
 'floating_visit_rate_1020_avg',
 'floating_visit_rate_1020_std',
 'floating_resd_rate_1020_gap',
 'floating_resd_rate_1020_gap_

In [169]:
gap_rate = []
for variable in col_list:
    if variable.find('gap_rate') > 0:
        gap_rate.append(variable)
    else:
        pass
    
train_data.drop(['em_g_gap_rate', 'resid_reg_pop_gap_rate', 'foreign_pop_gap_rate', 'total_pop_gap_rate', 'f_resid_reg_rate_gap_rate',\
 'f_foreign_rate_gap_rate', 'f_total_rate_gap_rate', 'floating_resd_rate_gap_rate', 'floating_pop_cnt_gap_rate',\
 'floating_visit_rate_1020_gap_rate', 'floating_resd_rate_1020_gap_rate', 'floating_visit_rate_60up_gap_rate',\
'floating_resd_rate_60up_gap_rate', 'floating_f_visit_rate_gap_rate', 'floating_f_resd_rate_gap_rate', 'visit_pop_cnt_gap_rate_sf',\
 'rewo_pop_cnt_CVU_lf_gap_rate', 'rewo_pop_cnt_ETC_lf_gap_rate', 'visit_pop_cnt_CVU_lf_gap_rate', 'visit_pop_cnt_ETC_lf_gap_rate',\
 'floating_pop_cnt_CVU_lf_gap_rate', 'floating_pop_cnt_ETC_lf_gap_rate', 'floating_pop_cnt_lf_gap_rate', 'visit_rate_CVU_lf_gap_rate',\
 'visit_rate_ETC_lf_gap_rate', 'agri_use_amt_gap_rate', 'mart_use_amt_gap_rate', 'deli_use_amt_gap_rate', 'food_use_amt_gap_rate',\
 'agri_per_use_amt_gap_rate', 'mart_per_use_amt_gap_rate', 'deli_per_use_amt_gap_rate', 'food_per_use_amt_gap_rate'], axis=1, inplace=True)

test_data.drop(['em_g_gap_rate', 'resid_reg_pop_gap_rate', 'foreign_pop_gap_rate', 'total_pop_gap_rate', 'f_resid_reg_rate_gap_rate',\
 'f_foreign_rate_gap_rate', 'f_total_rate_gap_rate', 'floating_resd_rate_gap_rate', 'floating_pop_cnt_gap_rate',\
 'floating_visit_rate_1020_gap_rate', 'floating_resd_rate_1020_gap_rate', 'floating_visit_rate_60up_gap_rate',\
'floating_resd_rate_60up_gap_rate', 'floating_f_visit_rate_gap_rate', 'floating_f_resd_rate_gap_rate', 'visit_pop_cnt_gap_rate_sf',\
 'rewo_pop_cnt_CVU_lf_gap_rate', 'rewo_pop_cnt_ETC_lf_gap_rate', 'visit_pop_cnt_CVU_lf_gap_rate', 'visit_pop_cnt_ETC_lf_gap_rate',\
 'floating_pop_cnt_CVU_lf_gap_rate', 'floating_pop_cnt_ETC_lf_gap_rate', 'floating_pop_cnt_lf_gap_rate', 'visit_rate_CVU_lf_gap_rate',\
 'visit_rate_ETC_lf_gap_rate', 'agri_use_amt_gap_rate', 'mart_use_amt_gap_rate', 'deli_use_amt_gap_rate', 'food_use_amt_gap_rate',\
 'agri_per_use_amt_gap_rate', 'mart_per_use_amt_gap_rate', 'deli_per_use_amt_gap_rate', 'food_per_use_amt_gap_rate'], axis=1, inplace=True)

In [170]:
std = []
for variable in col_list:
    if variable.find('std') > 0:
        std.append(variable)
    else:
        pass

train_data.drop(['em_g_std', 'resid_reg_pop_std', 'foreign_pop_std', 'total_pop_std', 'f_resid_reg_rate_std', 'f_foreign_rate_std',\
 'f_total_rate_std', 'floating_resd_rate_std', 'floating_pop_cnt_std', 'floating_visit_rate_1020_std',\
 'floating_resd_rate_1020_std', 'floating_visit_rate_60up_std', 'floating_resd_rate_60up_std', 'floating_f_visit_rate_std',\
 'floating_f_resd_rate_std', 'visit_pop_cnt_std_sf', 'rewo_pop_cnt_CVU_lf_std',\
 'rewo_pop_cnt_ETC_lf_std', 'visit_pop_cnt_CVU_lf_std', 'visit_pop_cnt_ETC_lf_std',\
 'floating_pop_cnt_CVU_lf_std', 'floating_pop_cnt_ETC_lf_std', 'floating_pop_cnt_lf_std', 'visit_rate_CVU_lf_std', \
'visit_rate_ETC_lf_std', 'agri_use_amt_std', 'mart_use_amt_std', 'deli_use_amt_std', 'food_use_amt_std',\
 'agri_per_use_amt_std', 'mart_per_use_amt_std', 'deli_per_use_amt_std', 'food_per_use_amt_std'], axis=1, inplace=True)

test_data.drop(['em_g_std', 'resid_reg_pop_std', 'foreign_pop_std', 'total_pop_std', 'f_resid_reg_rate_std', 'f_foreign_rate_std',\
 'f_total_rate_std', 'floating_resd_rate_std', 'floating_pop_cnt_std', 'floating_visit_rate_1020_std',\
 'floating_resd_rate_1020_std', 'floating_visit_rate_60up_std', 'floating_resd_rate_60up_std', 'floating_f_visit_rate_std',\
 'floating_f_resd_rate_std', 'visit_pop_cnt_std_sf', 'rewo_pop_cnt_CVU_lf_std',\
 'rewo_pop_cnt_ETC_lf_std', 'visit_pop_cnt_CVU_lf_std', 'visit_pop_cnt_ETC_lf_std',\
 'floating_pop_cnt_CVU_lf_std', 'floating_pop_cnt_ETC_lf_std', 'floating_pop_cnt_lf_std', 'visit_rate_CVU_lf_std', \
'visit_rate_ETC_lf_std', 'agri_use_amt_std', 'mart_use_amt_std', 'deli_use_amt_std', 'food_use_amt_std',\
 'agri_per_use_amt_std', 'mart_per_use_amt_std', 'deli_per_use_amt_std', 'food_per_use_amt_std'], axis=1, inplace=True)

# 4. 추가 변수 제거 

## 4.1 후진 제거법으로 아이디어를 얻어서 제거한 변수들 

- food_per_use_amt_avg
- food_per_use_amt_gap
- agri_use_amt_gap
- visit_rate_CVU_lf_avg
- floating_pop_cnt_lf_avg
- floating_pop_cnt_lf_gap
- floating_pop_cnt_CVU_lf_avg
- rewo_pop_cnt_CVU_lf_gap
- floating_f_resd_rate_gap
- floating_resd_rate_60up_avg
- f_total_rate_avg

# 5. 스케일링 및 X_train, y_train, X_test, y_test 분리

In [186]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

In [187]:
X_num_train = train_data[[ 'em_g_gap', 'em_g_avg', 'resid_reg_pop_gap','resid_reg_pop_avg', 'total_pop_gap', 'total_pop_avg',
       'f_resid_reg_rate_gap', 'f_resid_reg_rate_avg', 'f_foreign_rate_gap',
       'f_foreign_rate_avg', 'f_total_rate_gap', 'floating_resd_rate_gap', 'floating_resd_rate_avg',
       'floating_pop_cnt_gap', 'floating_pop_cnt_avg', 'floating_visit_rate_1020_gap', 'floating_visit_rate_1020_avg',
       'floating_resd_rate_1020_gap', 'floating_resd_rate_1020_avg','floating_visit_rate_60up_gap', 'floating_visit_rate_60up_avg',
       'floating_resd_rate_60up_gap',   'floating_f_visit_rate_gap', 'floating_f_visit_rate_avg',
        'floating_f_resd_rate_avg','visit_pop_cnt', 'visit_pop_cnt_gap_sf', 'visit_pop_cnt_avg_sf',  'rewo_pop_cnt_CVU_lf_avg',
       'rewo_pop_cnt_ETC_lf_gap', 'rewo_pop_cnt_ETC_lf_avg', 'visit_pop_cnt_CVU_lf_gap', 'visit_pop_cnt_CVU_lf_avg',
       'visit_pop_cnt_ETC_lf_gap', 'visit_pop_cnt_ETC_lf_avg', 'floating_pop_cnt_CVU_lf_gap','floating_pop_cnt_ETC_lf_gap', 
        'floating_pop_cnt_ETC_lf_avg', 'visit_rate_CVU_lf_gap','visit_rate_ETC_lf_gap', 'visit_rate_ETC_lf_avg',
       'agri_use_amt_avg', 'mart_use_amt_gap', 'mart_use_amt_avg', 'deli_use_amt_gap', 'deli_use_amt_avg', 'food_use_amt_gap',
       'food_use_amt_avg', 'agri_per_use_amt_gap', 'agri_per_use_amt_avg', 'mart_per_use_amt_gap', 'mart_per_use_amt_avg', 
        'deli_per_use_amt_gap','deli_per_use_amt_avg', 'temperature', 'rain', 'fine_dust','peak']]

scaler = StandardScaler()
scaler.fit(X_num_train)
X_train = scaler.transform(X_num_train)
X_train = pd.DataFrame(X_train, index=X_num_train.index, columns=X_num_train.columns)

y_train = train_data['em_g']

In [188]:
X_num_test = test_data[[ 'em_g_gap', 'em_g_avg', 'resid_reg_pop_gap','resid_reg_pop_avg', 'total_pop_gap', 'total_pop_avg',
       'f_resid_reg_rate_gap', 'f_resid_reg_rate_avg', 'f_foreign_rate_gap',
       'f_foreign_rate_avg', 'f_total_rate_gap', 'floating_resd_rate_gap', 'floating_resd_rate_avg',
       'floating_pop_cnt_gap', 'floating_pop_cnt_avg', 'floating_visit_rate_1020_gap', 'floating_visit_rate_1020_avg',
       'floating_resd_rate_1020_gap', 'floating_resd_rate_1020_avg','floating_visit_rate_60up_gap', 'floating_visit_rate_60up_avg',
       'floating_resd_rate_60up_gap',   'floating_f_visit_rate_gap', 'floating_f_visit_rate_avg',
        'floating_f_resd_rate_avg','visit_pop_cnt', 'visit_pop_cnt_gap_sf', 'visit_pop_cnt_avg_sf',  'rewo_pop_cnt_CVU_lf_avg',
       'rewo_pop_cnt_ETC_lf_gap', 'rewo_pop_cnt_ETC_lf_avg', 'visit_pop_cnt_CVU_lf_gap', 'visit_pop_cnt_CVU_lf_avg',
       'visit_pop_cnt_ETC_lf_gap', 'visit_pop_cnt_ETC_lf_avg', 'floating_pop_cnt_CVU_lf_gap','floating_pop_cnt_ETC_lf_gap', 
        'floating_pop_cnt_ETC_lf_avg', 'visit_rate_CVU_lf_gap','visit_rate_ETC_lf_gap', 'visit_rate_ETC_lf_avg',
       'agri_use_amt_avg', 'mart_use_amt_gap', 'mart_use_amt_avg', 'deli_use_amt_gap', 'deli_use_amt_avg', 'food_use_amt_gap',
       'food_use_amt_avg', 'agri_per_use_amt_gap', 'agri_per_use_amt_avg', 'mart_per_use_amt_gap', 'mart_per_use_amt_avg', 
        'deli_per_use_amt_gap','deli_per_use_amt_avg', 'temperature', 'rain', 'fine_dust','peak']]

scaler = StandardScaler()
scaler.fit(X_num_test)
X_test = scaler.transform(X_num_test)
X_test = pd.DataFrame(X_test, index=X_num_test.index, columns=X_num_test.columns)

C:\Users\gram_\anaconda3\envs\min\lib\site-packages\sklearn\utils\extmath.py:770: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\gram_\anaconda3\envs\min\lib\site-packages\sklearn\utils\extmath.py:711: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


# 6. 학습

## 6.1 LGBM

In [189]:
from lightgbm import LGBMRegressor
model_LGBM = LGBMRegressor(num_iterations=1000)

model_LGBM.fit(X_train, y_train)

C:\Users\gram_\anaconda3\envs\min\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMRegressor(num_iterations=1000)

## 6.2 XGB

In [190]:
from xgboost import XGBRegressor

model_XGB = XGBRegressor(learning_rate=0.125, n_estimators=300, max_depth=2)

model_XGB.fit(X_train, y_train)

[22:10:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.125, max_depth=2, n_estimators=300)

## 6.3 Voting 앙상블 학습

In [191]:
from sklearn.ensemble import VotingRegressor

voting_clf = VotingRegressor(estimators=[('lgbm1',model_LGBM), ('lgbm2',model_LGBM),  ('xgb',model_XGB),('lgbm',model_LGBM)])

voting_clf.fit(X_train, y_train)

pred = voting_clf.predict(X_test)

C:\Users\gram_\anaconda3\envs\min\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[22:10:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# 2021년 7월 8월 음식물 쓰레기 배출량 예측 

In [192]:
pred = pred.round(1)

In [193]:
pred

array([31425739.2, 52127490.4, 23626145. , 71813973.7, 31071661.8,
       52733670.4, 22489190.3, 74075356.3])

In [194]:
submission['em_g_7'] = pred[:4]
submission['em_g_8'] = pred[4:]

In [195]:
submission

,읍동면,em_g_7,em_g_8
0,구좌읍,31425739.2,31071661.8
1,조천읍,52127490.4,52733670.4
2,한경면,23626145.0,22489190.3
3,한림읍,71813973.7,74075356.3


In [181]:
submission.to_csv('그룹2_7월8월_음식물쓰레기예측.csv', index=False)